# ادغام Azure AI Search با NVIDIA NIM و LlamaIndex

در این دفترچه، نحوه استفاده از مدل‌های هوش مصنوعی NVIDIA و LlamaIndex برای ایجاد یک خط لوله قدرتمند Retrieval-Augmented Generation (RAG) را نشان خواهیم داد. ما از مدل‌های زبانی بزرگ و تعبیه‌های NVIDIA استفاده می‌کنیم، آن‌ها را با Azure AI Search به عنوان ذخیره‌سازی برداری ادغام می‌کنیم و RAG را برای بهبود کیفیت و کارایی جستجو انجام می‌دهیم.

## مزایا
- **مقیاس‌پذیری**: استفاده از مدل‌های زبانی بزرگ NVIDIA و Azure AI Search برای بازیابی مقیاس‌پذیر و کارآمد.
- **صرفه‌جویی در هزینه**: بهینه‌سازی جستجو و بازیابی با ذخیره‌سازی برداری کارآمد و تکنیک‌های جستجوی ترکیبی.
- **عملکرد بالا**: ترکیب مدل‌های زبانی قدرتمند با جستجوی برداری برای پاسخ‌های سریع‌تر و دقیق‌تر.
- **کیفیت**: حفظ کیفیت بالای جستجو با استناد به اسناد مرتبط بازیابی‌شده در پاسخ‌های مدل‌های زبانی.

## پیش‌نیازها
- 🐍 پایتون نسخه 3.9 یا بالاتر
- 🔗 [خدمات Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 کلید API NVIDIA برای دسترسی به مدل‌های زبانی بزرگ و تعبیه‌های NVIDIA از طریق میکروسرویس‌های NVIDIA NIM

## ویژگی‌های پوشش داده‌شده
- ✅ ادغام مدل‌های زبانی بزرگ NVIDIA (ما از [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) استفاده خواهیم کرد)
- ✅ تعبیه‌های NVIDIA (ما از [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) استفاده خواهیم کرد)
- ✅ حالت‌های پیشرفته بازیابی Azure AI Search
- ✅ ایندکس‌گذاری اسناد با LlamaIndex
- ✅ RAG با استفاده از Azure AI Search و LlamaIndex همراه با مدل‌های زبانی بزرگ NVIDIA

بزن بریم!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## نصب و نیازمندی‌ها  
یک محیط پایتون با استفاده از نسخه پایتون >3.10 ایجاد کنید.

## شروع به کار!  


برای شروع، شما به یک `NVIDIA_API_KEY` نیاز دارید تا بتوانید از مدل‌های NVIDIA AI Foundation استفاده کنید:
1) یک حساب کاربری رایگان در [NVIDIA](https://build.nvidia.com/explore/discover) ایجاد کنید.
2) روی مدل مورد نظر خود کلیک کنید.
3) در بخش ورودی، زبانه Python را انتخاب کنید و سپس روی **Get API Key** کلیک کنید و بعد **Generate Key** را فشار دهید.
4) کلید تولید شده را کپی کرده و به عنوان NVIDIA_API_KEY ذخیره کنید. از اینجا به بعد، شما باید به نقاط پایانی دسترسی داشته باشید.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## مثال RAG با استفاده از LLM و Embedding
### 1) مقداردهی اولیه LLM
`llama-index-llms-nvidia`، که به عنوان اتصال‌دهنده LLM انویدیا نیز شناخته می‌شود، به شما امکان می‌دهد به مدل‌های سازگار موجود در کاتالوگ API انویدیا متصل شوید و از آن‌ها خروجی بگیرید. برای مشاهده لیست مدل‌های تکمیل مکالمه، به اینجا مراجعه کنید: https://build.nvidia.com/search?term=Text-to-Text

در اینجا از **mixtral-8x7b-instruct-v0.1** استفاده خواهیم کرد.


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) مقداردهی اولیه به جاسازی

`llama-index-embeddings-nvidia` که به عنوان اتصال‌دهنده جاسازی‌های NVIDIA نیز شناخته می‌شود، به شما امکان می‌دهد به مدل‌های سازگار موجود در کاتالوگ API NVIDIA متصل شوید و از آن‌ها تولید کنید. ما مدل جاسازی `nvidia/nv-embedqa-e5-v5` را انتخاب کردیم. فهرستی از مدل‌های جاسازی متن را می‌توانید در اینجا مشاهده کنید: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### ۳) ایجاد یک ذخیره‌سازی برداری جستجوی Azure AI


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) ایجاد یک موتور جستجو برای پرسیدن سوالات درباره داده‌های شما

در اینجا یک جستجو با استفاده از جستجوی برداری خالص در Azure AI Search و اتصال پاسخ به مدل زبان بزرگ ما (Phi-3.5-MOE) آورده شده است.


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

اینجا یک پرسش با استفاده از جستجوی ترکیبی در Azure AI Search است.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### تحلیل جستجوی برداری
پاسخ LLM به‌طور دقیق پیامدهای اقتصادی کلیدی ذکر شده در متن منبع درباره بازار سهام روسیه را بیان می‌کند. به‌طور خاص، اشاره می‌کند که بازار سهام روسیه کاهش قابل توجهی داشته و ۴۰٪ از ارزش خود را از دست داده است و معاملات به دلیل وضعیت جاری متوقف شده‌اند. این پاسخ به‌خوبی با اطلاعات ارائه‌شده در منبع همخوانی دارد و نشان می‌دهد که LLM جزئیات مرتبط با تأثیر بازار سهام را به‌درستی شناسایی و خلاصه کرده است، که نتیجه اقدامات روسیه و تحریم‌های اعمال‌شده است.

#### توضیحات گره‌های منبع
گره‌های منبع شرح دقیقی از پیامدهای اقتصادی که روسیه به دلیل تحریم‌های بین‌المللی با آن مواجه شده است ارائه می‌دهند. متن تأکید می‌کند که بازار سهام روسیه ۴۰٪ از ارزش خود را از دست داده و معاملات متوقف شده‌اند. علاوه بر این، به سایر پیامدهای اقتصادی مانند کاهش ارزش روبل و انزوای گسترده‌تر اقتصاد روسیه اشاره می‌کند. پاسخ LLM به‌طور مؤثر نکات کلیدی این گره‌ها را استخراج کرده و بر تأثیر بازار سهام تمرکز کرده است، همان‌طور که در پرسش درخواست شده بود.


حالا بیایید به یک پرسش نگاه کنیم که در آن جستجوی هیبریدی پاسخ مستدل و مناسبی ارائه نمی‌دهد:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### جستجوی ترکیبی: تحلیل پاسخ LLM
پاسخ LLM در مثال جستجوی ترکیبی نشان می‌دهد که متن ارائه‌شده تاریخ دقیق حمله روسیه به اوکراین را مشخص نمی‌کند. این پاسخ نشان‌دهنده این است که LLM از اطلاعات موجود در اسناد منبع استفاده می‌کند اما به نبود جزئیات دقیق در متن اذعان دارد.

پاسخ به‌درستی تشخیص می‌دهد که متن به رویدادهای مرتبط با تجاوز روسیه اشاره دارد اما تاریخ دقیق حمله را مشخص نمی‌کند. این موضوع توانایی LLM در درک اطلاعات ارائه‌شده را نشان می‌دهد، در حالی که شکاف‌های موجود در محتوا را نیز شناسایی می‌کند. LLM به‌طور مؤثر کاربر را ترغیب می‌کند تا برای یافتن تاریخ دقیق به منابع خارجی یا سوابق تاریخی مراجعه کند و سطحی از احتیاط را در مواجهه با اطلاعات ناقص به نمایش می‌گذارد.

### تحلیل گره‌های منبع
گره‌های منبع در مثال جستجوی ترکیبی شامل بخش‌هایی از یک سخنرانی هستند که به واکنش ایالات متحده نسبت به اقدامات روسیه در اوکراین می‌پردازد. این گره‌ها بر تأثیرات گسترده‌تر ژئوپلیتیکی و اقداماتی که ایالات متحده و متحدانش در واکنش به حمله انجام داده‌اند تأکید دارند، اما تاریخ دقیق حمله را ذکر نمی‌کنند. این موضوع با پاسخ LLM همخوانی دارد که به‌درستی تشخیص می‌دهد متن فاقد اطلاعات دقیق درباره تاریخ حمله است.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### تحلیل ترکیبی با رتبه‌بندی مجدد: بررسی پاسخ LLM  
در مثال ترکیبی با رتبه‌بندی مجدد، پاسخ LLM با اشاره به اینکه رویداد شش روز قبل از سخنرانی رخ داده است، زمینه اضافی ارائه می‌دهد. این نشان می‌دهد که LLM قادر است تاریخ حمله را بر اساس زمان سخنرانی استنباط کند، حتی اگر برای دقت بیشتر همچنان نیاز به دانستن تاریخ دقیق سخنرانی داشته باشد.  

این پاسخ توانایی بهبود یافته‌ای را در استفاده از نشانه‌های زمینه‌ای برای ارائه پاسخی اطلاعاتی‌تر نشان می‌دهد. این مزیت رتبه‌بندی مجدد را برجسته می‌کند، جایی که LLM می‌تواند به اطلاعات مرتبط‌تر دسترسی پیدا کند و آن‌ها را اولویت‌بندی کند تا تقریب دقیق‌تری از جزئیات مورد نظر (یعنی تاریخ حمله) ارائه دهد.  

### تحلیل گره‌های منبع  
گره‌های منبع در این مثال شامل ارجاعاتی به زمان حمله روسیه هستند که به طور خاص اشاره می‌کنند که این حمله شش روز قبل از سخنرانی رخ داده است. اگرچه تاریخ دقیق هنوز به طور صریح بیان نشده است، گره‌ها زمینه زمانی ارائه می‌دهند که به LLM اجازه می‌دهد پاسخی دقیق‌تر و ظریف‌تر ارائه دهد. گنجاندن این جزئیات نشان می‌دهد که چگونه رتبه‌بندی مجدد می‌تواند توانایی LLM را در استخراج و استنباط اطلاعات از زمینه ارائه‌شده بهبود بخشد و در نتیجه پاسخی دقیق‌تر و اطلاعاتی‌تر ارائه دهد.  


توجه:  
در این دفترچه، ما از میکروسرویس‌های NVIDIA NIM از کاتالوگ API‌های NVIDIA استفاده کرده‌ایم.  
API‌های ذکر شده، `NVIDIA (llms)`، `NVIDIAEmbedding` و [Azure AI Search Semantic Hybrid Retrieval (با قابلیت رتبه‌بندی داخلی)](https://learn.microsoft.com/azure/search/semantic-search-overview) هستند. توجه داشته باشید که API‌های فوق همچنین می‌توانند از میکروسرویس‌های خود میزبان پشتیبانی کنند.  

**مثال:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
